In [1]:
import tensorflow as tf
import pandas as pd   ## data analysis and manipulation
import numpy as np    ## numerial computing
import seaborn as sns ##  data visualization library based on matplotlib
import tensorflow.keras as keras

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.metrics import average_precision_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

## Data importing

In [3]:
df = pd.read_excel("/Users/a123456/Desktop/Fei's Project/sf removed,negrativeFN removed/oct_no_namev3.xlsx")
df

,Unnamed: 0.1,Unnamed: 0,DOB,Eye,Scan.Type,Diameter..mm.,Diameter....,Fixed.in.mm,ExamDate,ExamTime,...,RNFLT.761,RNFLT.762,RNFLT.763,RNFLT.764,RNFLT.765,RNFLT.766,RNFLT.767,RNFLT.768,freq,patient_id
0,0,0,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2014-05-06,10:00:00 (UTC),...,45.0,45.0,45.0,45.0,46.0,46.0,47.0,48.0,11,LA1001
1,1,1,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2014-11-06,10:16:54 (UTC),...,56.0,56.0,57.0,57.0,58.0,58.0,59.0,59.0,11,LA1001
2,2,2,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2015-08-20,10:58:05 (UTC),...,51.0,52.0,53.0,53.0,54.0,54.0,55.0,55.0,11,LA1001
3,3,3,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2016-02-25,08:32:37 (UTC),...,50.0,50.0,51.0,51.0,52.0,53.0,54.0,55.0,11,LA1001
4,4,4,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2016-07-15,08:43:16 (UTC),...,49.0,50.0,51.0,52.0,52.0,54.0,55.0,56.0,11,LA1001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,2500,2500,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2015-10-06,12:42:31 (UTC),...,34.0,34.0,34.0,33.0,33.0,33.0,33.0,33.0,9,LA1146
2501,2501,2501,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2016-04-05,12:50:37 (UTC),...,33.0,32.0,31.0,31.0,31.0,31.0,31.0,31.0,9,LA1146
2502,2502,2502,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2017-11-03,14:20:22 (UTC),...,33.0,33.0,32.0,32.0,32.0,33.0,33.0,34.0,9,LA1146
2503,2503,2503,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2018-05-02,10:19:29 (UTC),...,31.0,31.0,30.0,30.0,30.0,30.0,30.0,31.0,9,LA1146


In [4]:
df.shape

(2505, 797)

In [5]:
pd.get_dummies(df.iloc[:, 4]).value_counts()#The result means all records were from OCT Circle Scan

OCT Circle Scan
1                  2505
dtype: int64

In [6]:
df.iloc[:, 795].value_counts()
#The min freq is 5, and the max one is 23

13    468
12    408
14    280
10    180
11    165
9     162
7     147
5     130
6     114
15    105
8     104
17     51
16     48
23     46
21     42
18     36
19     19
Name: freq, dtype: int64

In [7]:
#read in the VF_progression
output_df = pd.read_csv("/Users/a123456/Desktop/Fei's Project/sf removed,negrativeFN removed/VF_Progression.csv")
output_df

,Eye_ID,MD_Baseline,MD_Slope,MD_P,VFI_Baseline,VFI_Slope,VFI_P,GRI,GRI_Decay,GRI_Imp,GRI_NC,PLR__-1__0.01,PoPLR__-1__0.01,ExamDate_Baseline,ExamDate_Final,Number_Exams,Follow_Up,Follow_Up_Check
0,LA1001_LE,-2.86,-0.280014,0.293638,99,-0.441584,0.198784,-7.238855,11,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930
1,LA1001_RE,-5.95,-3.354522,0.003489,95,-12.966853,0.001769,-100.000000,44,0,8,36,33,2014-05-06,2020-01-21,8,5.711157,5.711157
2,LA1002_LE,-0.30,0.167083,0.295050,99,0.205077,0.377241,0.970795,0,3,49,0,0,2015-02-11,2020-11-25,9,5.787817,5.787817
3,LA1002_RE,-10.69,0.326495,0.146388,73,0.806200,0.153700,5.615068,0,4,48,0,0,2015-02-11,2020-11-25,9,5.787817,5.787817
4,LA1004_LE,-1.72,0.151676,0.356400,99,-0.666145,0.018454,3.863507,1,8,43,1,1,2013-11-18,2018-01-24,9,4.183436,4.183436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,LA1199_RE,-16.91,-0.331580,0.063464,61,-2.653492,0.002033,-10.111738,10,4,38,4,3,2013-10-23,2020-06-03,10,6.611910,6.611910
256,LA1200_LE,-3.48,-1.626588,0.020859,97,-4.635423,0.019451,-18.479868,9,1,42,10,10,2013-06-11,2019-10-23,10,6.365503,6.365503
257,LA1200_RE,-3.34,-0.746755,0.033740,93,-3.257121,0.001076,-15.791907,7,0,45,3,3,2013-06-11,2018-10-31,11,5.388090,5.388090
258,LA1201_LE,-6.81,-0.148221,0.147443,91,-0.746638,0.094541,-3.535039,4,0,48,2,1,2013-03-13,2020-08-26,13,7.455168,7.455168


In [8]:
len(output_df)

260

In [9]:
output_df['y_GRI'] = pd.Series(np.select([np.less(output_df["GRI"], -6)],[1],0))
output_df

,Eye_ID,MD_Baseline,MD_Slope,MD_P,VFI_Baseline,VFI_Slope,VFI_P,GRI,GRI_Decay,GRI_Imp,GRI_NC,PLR__-1__0.01,PoPLR__-1__0.01,ExamDate_Baseline,ExamDate_Final,Number_Exams,Follow_Up,Follow_Up_Check,y_GRI
0,LA1001_LE,-2.86,-0.280014,0.293638,99,-0.441584,0.198784,-7.238855,11,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930,1
1,LA1001_RE,-5.95,-3.354522,0.003489,95,-12.966853,0.001769,-100.000000,44,0,8,36,33,2014-05-06,2020-01-21,8,5.711157,5.711157,1
2,LA1002_LE,-0.30,0.167083,0.295050,99,0.205077,0.377241,0.970795,0,3,49,0,0,2015-02-11,2020-11-25,9,5.787817,5.787817,0
3,LA1002_RE,-10.69,0.326495,0.146388,73,0.806200,0.153700,5.615068,0,4,48,0,0,2015-02-11,2020-11-25,9,5.787817,5.787817,0
4,LA1004_LE,-1.72,0.151676,0.356400,99,-0.666145,0.018454,3.863507,1,8,43,1,1,2013-11-18,2018-01-24,9,4.183436,4.183436,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,LA1199_RE,-16.91,-0.331580,0.063464,61,-2.653492,0.002033,-10.111738,10,4,38,4,3,2013-10-23,2020-06-03,10,6.611910,6.611910,1
256,LA1200_LE,-3.48,-1.626588,0.020859,97,-4.635423,0.019451,-18.479868,9,1,42,10,10,2013-06-11,2019-10-23,10,6.365503,6.365503,1
257,LA1200_RE,-3.34,-0.746755,0.033740,93,-3.257121,0.001076,-15.791907,7,0,45,3,3,2013-06-11,2018-10-31,11,5.388090,5.388090,1
258,LA1201_LE,-6.81,-0.148221,0.147443,91,-0.746638,0.094541,-3.535039,4,0,48,2,1,2013-03-13,2020-08-26,13,7.455168,7.455168,0


out = output_df.values.tolist()
for i in range(len(output_df)):
    out[i][0] = out[i][0].replace("LE", "L")#Change name from LE to L
    out[i][0] = out[i][0].replace("RE", "R")#Change name from RE to R
out

out_df = pd.DataFrame(out, columns = ['Eye_ID', 'MD_Baseline', 'MD_Slope', 'MD_P', 'VFI_Baseline',
       'VFI_Slope', 'VFI_P', 'GRI', 'GRI_Decay', 'GRI_Imp', 'GRI_NC',
       'PLR__-1__0.01', 'PoPLR__-1__0.01', 'ExamDate_Baseline',
       'ExamDate_Final', 'Number_Exams', 'Follow_Up', 'Follow_Up_Check'])
out_df

In [10]:
df["com_id"] = df.values[:,796]+"_"+df.values[:,3]+"E" #Create a new column used to join the output_df
df

,Unnamed: 0.1,Unnamed: 0,DOB,Eye,Scan.Type,Diameter..mm.,Diameter....,Fixed.in.mm,ExamDate,ExamTime,...,RNFLT.762,RNFLT.763,RNFLT.764,RNFLT.765,RNFLT.766,RNFLT.767,RNFLT.768,freq,patient_id,com_id
0,0,0,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2014-05-06,10:00:00 (UTC),...,45.0,45.0,45.0,46.0,46.0,47.0,48.0,11,LA1001,LA1001_LE
1,1,1,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2014-11-06,10:16:54 (UTC),...,56.0,57.0,57.0,58.0,58.0,59.0,59.0,11,LA1001,LA1001_LE
2,2,2,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2015-08-20,10:58:05 (UTC),...,52.0,53.0,53.0,54.0,54.0,55.0,55.0,11,LA1001,LA1001_LE
3,3,3,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2016-02-25,08:32:37 (UTC),...,50.0,51.0,51.0,52.0,53.0,54.0,55.0,11,LA1001,LA1001_LE
4,4,4,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2016-07-15,08:43:16 (UTC),...,50.0,51.0,52.0,52.0,54.0,55.0,56.0,11,LA1001,LA1001_LE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,2500,2500,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2015-10-06,12:42:31 (UTC),...,34.0,34.0,33.0,33.0,33.0,33.0,33.0,9,LA1146,LA1146_RE
2501,2501,2501,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2016-04-05,12:50:37 (UTC),...,32.0,31.0,31.0,31.0,31.0,31.0,31.0,9,LA1146,LA1146_RE
2502,2502,2502,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2017-11-03,14:20:22 (UTC),...,33.0,32.0,32.0,32.0,33.0,33.0,34.0,9,LA1146,LA1146_RE
2503,2503,2503,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2018-05-02,10:19:29 (UTC),...,31.0,30.0,30.0,30.0,30.0,30.0,31.0,9,LA1146,LA1146_RE


In [11]:
df_mer = pd.DataFrame.merge(df, output_df, left_on = "com_id", right_on = "Eye_ID") 
#Here we are using inner join, which means only the records are both in left table and right table will be kept.
df_mer

,Unnamed: 0.1,Unnamed: 0,DOB,Eye,Scan.Type,Diameter..mm.,Diameter....,Fixed.in.mm,ExamDate,ExamTime,...,GRI_Imp,GRI_NC,PLR__-1__0.01,PoPLR__-1__0.01,ExamDate_Baseline,ExamDate_Final,Number_Exams,Follow_Up,Follow_Up_Check,y_GRI
0,0,0,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2014-05-06,10:00:00 (UTC),...,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930,1
1,1,1,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2014-11-06,10:16:54 (UTC),...,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930,1
2,2,2,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2015-08-20,10:58:05 (UTC),...,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930,1
3,3,3,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2016-02-25,08:32:37 (UTC),...,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930,1
4,4,4,5/3/1946,L,OCT Circle Scan,3.8,12.0,0,2016-07-15,08:43:16 (UTC),...,0,41,1,1,2014-05-06,2020-09-22,8,6.381930,6.381930,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668,2500,2500,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2015-10-06,12:42:31 (UTC),...,0,34,3,2,2013-05-08,2020-01-22,9,6.707734,6.707734,1
1669,2501,2501,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2016-04-05,12:50:37 (UTC),...,0,34,3,2,2013-05-08,2020-01-22,9,6.707734,6.707734,1
1670,2502,2502,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2017-11-03,14:20:22 (UTC),...,0,34,3,2,2013-05-08,2020-01-22,9,6.707734,6.707734,1
1671,2503,2503,5/31/1939,R,OCT Circle Scan,3.5,12.0,0,2018-05-02,10:19:29 (UTC),...,0,34,3,2,2013-05-08,2020-01-22,9,6.707734,6.707734,1


In [12]:
df_mer.iloc[:, 796].value_counts()
#Length = 105 means there are totally 105 different patients.

LA1019    41
LA1122    35
LA1050    34
LA1054    31
LA1011    30
          ..
LA1137     5
LA1004     5
LA1141     5
LA1023     5
LA1075     5
Name: patient_id, Length: 105, dtype: int64

In [13]:
df_mer.iloc[:, 797].value_counts()
#Length = 160 means there are totally 160 different eyes.

LA1086_RE    23
LA1019_LE    23
LA1030_LE    21
LA1122_LE    19
LA1054_RE    18
             ..
LA1029_RE     5
LA1031_RE     5
LA1075_LE     5
LA1098_RE     5
LA1072_RE     5
Name: com_id, Length: 160, dtype: int64

In [14]:
df_mer.iloc[:, 816].value_counts()
#1124 non-progressing eye records, 549 progressing eye records

0    1124
1     549
Name: y_GRI, dtype: int64

In [17]:
y = df_mer.iloc[:, 816]
y

0       1
1       1
2       1
3       1
4       1
       ..
1668    1
1669    1
1670    1
1671    1
1672    1
Name: y_GRI, Length: 1673, dtype: int64

## Data splitting

In [16]:
# https://github.com/applied-systems-biology/Dynamic_SPHARM/blob/master/SPHARM/classes/stratified_group_shuffle_split.py

from __future__ import print_function
from __future__ import division

from sklearn.utils.validation import check_array
from sklearn.model_selection import StratifiedShuffleSplit


class GroupShuffleSplitStratified(StratifiedShuffleSplit):

    def __init__(self, n_splits=5, test_size=2, train_size=None, random_state=None):

        super(GroupShuffleSplitStratified, self).__init__(
            n_splits=n_splits,
            test_size=test_size,
            train_size=train_size,
            random_state=random_state)

    def _iter_indices(self, X, y, groups):
        if groups is None:
            raise ValueError("The 'groups' parameter should not be None.")
        groups = check_array(groups, ensure_2d=False, dtype=None)
        groups_unique, group_indices = np.unique(groups, return_inverse=True)
        classes = []
        for gr in groups_unique:
            classes.append(y[np.where(groups==gr)[0][0]])

        for group_train, group_test in super(
                GroupShuffleSplitStratified, self)._iter_indices(X=groups_unique, y=classes):
            # these are the indices of classes in the partition
            # invert them into data indices

            train = np.flatnonzero(np.in1d(group_indices, group_train))
            test = np.flatnonzero(np.in1d(group_indices, group_test))

            yield train, test

    def split(self, X, y=None, groups=None):
        return super(GroupShuffleSplitStratified, self).split(X, y, groups)

In [24]:
train_i,test_i = next(GroupShuffleSplitStratified(n_splits=2, test_size=0.1,
                                        random_state=8).split(df_mer,y, groups=df_mer['patient_id']))
TrainVal = df_mer.iloc[train_i]
TestSet = df_mer.iloc[test_i]
print(TrainVal.shape)
print(TestSet.shape)

(1507, 817)
(166, 817)


In [25]:
train_id,val_id = next(GroupShuffleSplitStratified(n_splits=2, test_size=0.1,
                                        random_state=8).split(TrainVal,y.iloc[train_i], groups=TrainVal['patient_id']))
TrainSet = TrainVal.iloc[train_id]
ValSet = TrainVal.iloc[val_id]
print(TrainSet.shape)
print(ValSet.shape)

(1349, 817)
(158, 817)


In [33]:
help(max)

Help on built-in function max in module builtins:

max(...)
    max(iterable, *[, default=obj, key=func]) -> value
    max(arg1, arg2, *args, *[, key=func]) -> value
    
    With a single iterable argument, return its biggest item. The
    default keyword-only argument specifies an object to return if
    the provided iterable is empty.
    With two or more arguments, return the largest argument.



In [37]:
max(df_mer.iloc[:, 27:795].max())
#The largest value for the whole dataframe is 382

382.0

In [48]:
x = df_mer.iloc[test_i, 27:795]
print(x.isnull().values.sum())
print(x.shape)
x.head()

768
(166, 768)


,RNFLT.1,RNFLT.2,RNFLT.3,RNFLT.4,RNFLT.5,RNFLT.6,RNFLT.7,RNFLT.8,RNFLT.9,RNFLT.10,...,RNFLT.759,RNFLT.760,RNFLT.761,RNFLT.762,RNFLT.763,RNFLT.764,RNFLT.765,RNFLT.766,RNFLT.767,RNFLT.768
149,52.0,52.0,52.0,52.0,53.0,53.0,53.0,53.0,53.0,54.0,...,52.0,52.0,52.0,52.0,52.0,52.0,52.0,51.0,51.0,51.0
150,48.0,48.0,48.0,49.0,49.0,49.0,49.0,50.0,50.0,51.0,...,49.0,49.0,48.0,48.0,48.0,48.0,48.0,48.0,48.0,48.0
151,44.0,44.0,45.0,45.0,46.0,46.0,46.0,46.0,46.0,46.0,...,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0
152,46.0,46.0,46.0,47.0,47.0,47.0,47.0,47.0,47.0,48.0,...,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0,46.0
153,50.0,50.0,51.0,51.0,51.0,52.0,52.0,52.0,51.0,51.0,...,50.0,49.0,49.0,49.0,48.0,48.0,48.0,49.0,49.0,50.0


In [49]:
x = np.asarray(x)
scaled_x = x/382
scaled_x = scaled_x.reshape(scaled_x.shape[0],scaled_x.shape[1])
X_test = scaled_x
print(X_test.shape)

(166, 768)


In [50]:
x = TrainVal.iloc[train_id, 27:795]
x = np.asarray(x)
scaled_x = x/382
scaled_x = scaled_x.reshape(scaled_x.shape[0],scaled_x.shape[1])
X_train = scaled_x
print(X_train.shape)

(1349, 768)


In [51]:
x = TrainVal.iloc[val_id, 27:795]
x = np.asarray(x)
scaled_x = x/382
scaled_x = scaled_x.reshape(scaled_x.shape[0],scaled_x.shape[1])
X_val = scaled_x
print(X_val.shape)

(158, 768)


In [52]:
#one-hot-encoding our label
y = pd.get_dummies(y)
y = y.rename(columns={0: "Non-Progressor", 1: "Progressor"})
print(y.value_counts())
y #The second column is 'progressor', The first column is 'non-progressor'

Non-Progressor  Progressor
1               0             1124
0               1              549
dtype: int64


,Non-Progressor,Progressor
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1668,0,1
1669,0,1
1670,0,1
1671,0,1


In [53]:
y_test = y.iloc[test_i]
y_test

,Non-Progressor,Progressor
149,1,0
150,1,0
151,1,0
152,1,0
153,1,0
...,...,...
1668,0,1
1669,0,1
1670,0,1
1671,0,1


In [54]:
y_train = y.iloc[train_i].iloc[train_id]
y_train

,Non-Progressor,Progressor
22,1,0
23,1,0
24,1,0
25,1,0
26,1,0
...,...,...
1649,1,0
1650,1,0
1651,1,0
1652,1,0


In [55]:
y_val = y.iloc[train_i].iloc[val_id]
y_val

,Non-Progressor,Progressor
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1541,1,0
1542,1,0
1543,1,0
1544,1,0


In [56]:
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.value_counts(), '\n')
print(y_val.value_counts(), '\n')
print(y_test.value_counts())

(1349, 768)
(166, 768)
(158, 768)
Non-Progressor  Progressor
1               0             915
0               1             434
dtype: int64 

Non-Progressor  Progressor
1               0             104
0               1              54
dtype: int64 

Non-Progressor  Progressor
1               0             105
0               1              61
dtype: int64


## Model building

In [15]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Flatten, TimeDistributed, LSTM

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape = (?, 768)))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()